In [85]:
import functools
import numpy as np
import tensorflow as tf

In [86]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [87]:
import requests

response = requests.get(TEST_DATA_URL)
#print(response.content)
# with open("test.csv", "wb") as f:
#     f.write(response.content)

In [46]:
np.set_printoptions(precision=3, suppress=True)

In [47]:
!tail {train_file_path}

1,female,15.0,0,0,7.225,Third,unknown,Cherbourg,y
0,male,20.0,0,0,9.8458,Third,unknown,Southampton,y
0,male,19.0,0,0,7.8958,Third,unknown,Southampton,y
0,male,28.0,0,0,7.8958,Third,unknown,Southampton,y
0,female,22.0,0,0,10.5167,Third,unknown,Southampton,y
0,male,28.0,0,0,10.5,Second,unknown,Southampton,y
0,male,25.0,0,0,7.05,Third,unknown,Southampton,y
1,female,19.0,0,0,30.0,First,B,Southampton,y
0,female,28.0,1,2,23.45,Third,unknown,Southampton,n
0,male,32.0,0,0,7.75,Third,unknown,Queenstown,y


In [18]:
import pandas as pd 

data = pd.read_csv(TRAIN_DATA_URL)
print(data)

     survived     sex   age  n_siblings_spouses  parch     fare   class  \
0           0    male  22.0                   1      0   7.2500   Third   
1           1  female  38.0                   1      0  71.2833   First   
2           1  female  26.0                   0      0   7.9250   Third   
3           1  female  35.0                   1      0  53.1000   First   
4           0    male  28.0                   0      0   8.4583   Third   
..        ...     ...   ...                 ...    ...      ...     ...   
622         0    male  28.0                   0      0  10.5000  Second   
623         0    male  25.0                   0      0   7.0500   Third   
624         1  female  19.0                   0      0  30.0000   First   
625         0  female  28.0                   1      2  23.4500   Third   
626         0    male  32.0                   0      0   7.7500   Third   

        deck  embark_town alone  
0    unknown  Southampton     n  
1          C    Cherbourg     n

In [48]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [88]:
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=5,
        label_name=LABEL_COLUMN,
        na_value='?',
        num_epochs=1,
        ignore_errors=True,
        **kwargs
    )
    return dataset

In [89]:
raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [90]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print("{:20s}: {}".format(key, value.numpy()))

In [91]:
show_batch(raw_train_data)

sex                 : [b'female' b'female' b'male' b'male' b'male']
age                 : [31. 60. 28. 28. 48.]
n_siblings_spouses  : [0 1 0 1 1]
parch               : [2 0 0 1 0]
fare                : [164.867  75.25    0.     15.246  76.729]
class               : [b'First' b'First' b'Second' b'Third' b'First']
deck                : [b'C' b'D' b'unknown' b'unknown' b'D']
embark_town         : [b'Southampton' b'Cherbourg' b'Southampton' b'Cherbourg' b'Cherbourg']
alone               : [b'n' b'n' b'y' b'n' b'n']


In [92]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch',
               'fare', 'class', 'deck', 'embark_town', 'alone']
tmp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)
show_batch(tmp_dataset)

sex                 : [b'female' b'male' b'male' b'female' b'female']
age                 : [28.  28.  30.5 27.  28. ]
n_siblings_spouses  : [1 0 0 0 0]
parch               : [0 0 0 0 0]
fare                : [16.1   0.    8.05 10.5  13.  ]
class               : [b'Third' b'Second' b'Third' b'Second' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'E' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'n' b'y' b'y' b'y' b'y']


In [93]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path,
                           select_columns=SELECT_COLUMNS, 
                           column_defaults=DEFAULTS)
show_batch(temp_dataset)

age                 : [34. 18. 28. 34. 28.]
n_siblings_spouses  : [1. 0. 0. 1. 1.]
parch               : [1. 0. 0. 0. 0.]
fare                : [ 14.4    7.75   7.75  26.   133.65]


In [94]:
example_batch, labels_batch = next(iter(temp_dataset))

In [95]:
def pack(features, label):
    return tf.stack(list(features.values()), axis=-1), label

In [96]:
packed_dataset = temp_dataset.map(pack)

In [97]:
for features, labels in packed_dataset.take(1):
    print(features.numpy())
    print()
    print(labels.numpy())

[[44.     1.     0.    26.   ]
 [70.     1.     1.    71.   ]
 [15.     0.     0.     7.225]
 [28.     0.     0.    13.   ]
 [32.     0.     0.    13.   ]]

[0 0 1 1 1]


In [98]:
class PackNumericFeatures(object):
    def __init__(self, names):
        self.names = names
        
    def __call__(self, features, labels):
        numeric_features = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features['numeric'] = numeric_features
        
        return features, labels

In [99]:
NUMERIC_FEATURES = ['age', 'n_siblings_spouses', 'parch', 'fare']
packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))
packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [100]:
show_batch(packed_train_data)

sex                 : [b'male' b'male' b'female' b'male' b'female']
class               : [b'Third' b'First' b'Third' b'First' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'B' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'y' b'n' b'y' b'n' b'n']
numeric             : [[ 28.      0.      0.      7.225]
 [ 50.      2.      0.    133.65 ]
 [ 22.      0.      0.      7.75 ]
 [ 31.      1.      0.     57.   ]
 [ 18.      1.      0.     17.8  ]]


In [101]:
example_batch, labels_batch = next(iter(packed_train_data))

In [102]:
example_batch

OrderedDict([('sex',
              <tf.Tensor: id=1269, shape=(5,), dtype=string, numpy=array([b'female', b'male', b'female', b'male', b'female'], dtype=object)>),
             ('class',
              <tf.Tensor: id=1265, shape=(5,), dtype=string, numpy=array([b'Third', b'Third', b'First', b'Third', b'Third'], dtype=object)>),
             ('deck',
              <tf.Tensor: id=1266, shape=(5,), dtype=string, numpy=array([b'G', b'unknown', b'E', b'unknown', b'unknown'], dtype=object)>),
             ('embark_town',
              <tf.Tensor: id=1267, shape=(5,), dtype=string, numpy=
              array([b'Southampton', b'Southampton', b'Cherbourg', b'Southampton',
                     b'Southampton'], dtype=object)>),
             ('alone',
              <tf.Tensor: id=1264, shape=(5,), dtype=string, numpy=array([b'n', b'n', b'y', b'n', b'n'], dtype=object)>),
             ('numeric',
              <tf.Tensor: id=1268, shape=(5, 4), dtype=float32, numpy=
              array([[ 2.   ,  0.

In [103]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [104]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [105]:
def normalize_numeric_data(data, mean, std):
    return (data - mean) / std

In [106]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

In [107]:
numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])

In [108]:
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7efbd622b840>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [109]:
example_batch['numeric']

<tf.Tensor: id=1268, shape=(5, 4), dtype=float32, numpy=
array([[ 2.   ,  0.   ,  1.   , 10.462],
       [30.   ,  1.   ,  0.   , 16.1  ],
       [30.   ,  0.   ,  0.   , 56.929],
       [ 9.   ,  5.   ,  2.   , 46.9  ],
       [38.   ,  1.   ,  5.   , 31.388]], dtype=float32)>

In [110]:
numeric_columns = [numeric_column]
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-2.208, -0.474,  0.782, -0.438],
       [ 0.029,  0.395, -0.479, -0.335],
       [ 0.029, -0.474, -0.479,  0.413],
       [-1.649,  3.87 ,  2.043,  0.229],
       [ 0.669,  0.395,  5.827, -0.055]], dtype=float32)

In [160]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class': ['First', 'Second', 'Third'],
    'deck': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    "embark_town": ['Cherbourg', 'Southampton', 'Queenstown'],
    'alone': ['y', 'n']
}

In [161]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [162]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [163]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy())

[[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1.]]


In [166]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns + numeric_columns)
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.     1.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     1.     0.     0.     0.     0.     1.     0.    -2.208 -0.474
  0.782 -0.438  0.     1.   ]


In [168]:
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimzier='adam',
    metrics=['accuracy'])

In [169]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [170]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 2s 13ms/step - loss: 0.4870 - accuracy: 0.7687
Epoch 2/20
126/126 [==============================] - 0s 4ms/step - loss: 0.4289 - accuracy: 0.8038
Epoch 3/20
126/126 [==============================] - 0s 4ms/step - loss: 0.4076 - accuracy: 0.8182
Epoch 4/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3976 - accuracy: 0.8341
Epoch 5/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3962 - accuracy: 0.8341
Epoch 6/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3851 - accuracy: 0.8421
Epoch 7/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3813 - accuracy: 0.8357
Epoch 8/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3699 - accuracy: 0.8517
Epoch 9/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3712 - accuracy: 0.8357
Epoch 10/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3665 - accuracy: 0.843

In [172]:
test_loss, test_accuracy = model.evaluate(test_data)
print('\n\n Test Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

53/53 [==============================] - 0s 6ms/step - loss: 0.4495 - accuracy: 0.8409


 Test Loss 0.44953719421096566, Test Accuracy 0.8409090638160706


In [176]:
predictions = model.predict(test_data)

for prediction, survived in zip(predictions, list(test_data)[0][1]):
    print("Predicted survival : {:2%}".format(prediction[0]),
          " | Actual outcome : ", "SURVIVED" if bool(survived) else "DIED")

Predicted survival : 7.144302%  | Actual outcome :  SURVIVED
Predicted survival : 3.872621%  | Actual outcome :  DIED
Predicted survival : 12.579799%  | Actual outcome :  DIED
Predicted survival : 31.808433%  | Actual outcome :  DIED
Predicted survival : 29.123947%  | Actual outcome :  SURVIVED
